In [2]:
#pip install selenium

In [3]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style

In [5]:
#打开网页
driver = webdriver.Chrome()
url = 'https://tj.esf.fang.com/house-a010383-b011784/'
driver.get(url)

In [8]:
string_list = []

i = 0

while i < 20:
    #爬取数据
    table = driver.find_element('class name','shop_list')
    rows = table.find_elements(By.TAG_NAME, 'dl')

    # 提取信息
    data = []
    for row in rows:
        title = row.find_element(By.CSS_SELECTOR, "h4 a").text
        details_text = row.find_element(By.CSS_SELECTOR, "p.tel_shop").text
        details = details_text.split("|")
        if len(details) >= 5:
            house_type = details[0]  
            area = details[1]         
            floor = details[2]        
            direction = details[3]    
            built_year = details[4]  
        else:
            house_type = area = floor = direction = built_year = None 
        name = row.find_element(By.CSS_SELECTOR, "p.add_shop a").text
        locate = row.find_element(By.CSS_SELECTOR, "p.add_shop span").text
        price_dd = row.find_element(By.CSS_SELECTOR, "dd.price_right")
        total_price = price_dd.find_element(By.CSS_SELECTOR, "span.red").text
        price_per_m2 = price_dd.find_element(By.CSS_SELECTOR, "span:not(.red)").text
        data.append([title, house_type, area, floor, direction, built_year, name, locate, total_price, price_per_m2])

    df = pd.DataFrame(data)
    #检测运行
    print(1)
    string_list.append(df)
    #翻页
    try:
        next_page=driver.find_element('class name','last')
        next_page.click()
        i+=1
    except NoSuchElementException:
        break

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [9]:
driver.quit()

In [10]:
#进一步收拾数据
df_full=pd.concat(string_list)
df_full.columns=['标题',
'房型',
'面积',
'层型',
'朝向','建成时间','小区名','区域','总价','单价']

In [11]:
df_full.describe()

,标题,房型,面积,层型,朝向,建成时间,小区名,区域,总价,单价
count,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200
unique,690,15,332,115,95,27,100,81,268,697
top,工抵房6套66万精装小高洋房总价低近车接车送靠谱,3室2厅,卧室：4个,无花园,南北向,2018年建,博屿海,中新生态城-生态城北区中新大道与中海大道交汇,130万,7586元/㎡
freq,16,570,67,167,823,217,82,82,36,12


In [12]:
#导出d
df_full.to_csv('price.csv', index=False)